# OBTAIN SAREK INPUT FILE FOR BLOOD, MELANOMA AND UPS SAMPLES

CREATE A CSV INPUT FILE FOR SAREK PIPELINE so it contains these structure:

patient,sex,status,sample,lane,fastq_1,fastq_2
patient1,XX,0,normal_sample,lane_1,test_L001_1.fastq.gz,test_L001_2.fastq.gz
patient1,XX,0,normal_sample,lane_2,test_L002_1.fastq.gz,test_L002_2.fastq.gz
patient1,XX,0,normal_sample,lane_3,test_L003_1.fastq.gz,test_L003_2.fastq.gz
patient1,XX,1,tumor_sample,lane_1,test2_L001_1.fastq.gz,test2_L001_2.fastq.gz
patient1,XX,1,tumor_sample,lane_2,test2_L002_1.fastq.gz,test2_L002_2.fastq.gz
patient1,XX,1,relapse_sample,lane_1,test3_L001_1.fastq.gz,test3_L001_2.fastq.gz

In [1]:
#Import pandas and read excel file with sequencing information
import pandas as pd
ruta = '/workspace/projects/sjd_melos/sequencing/LOPEBIG_55.xls'
metadata_df = pd.read_excel(ruta,skiprows= 2)
metadata_df.head()

,FLOWCELL,LANE,MULTIPLEX INDEX,APPLICATION,LIBRARY BARCODE (internal id),SAMPLE BARCODE,SAMPLE NAME,SPECIES,MACHINE,Unnamed: 9
0,HL3LCDSX7,1,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NB,HL3LCDSX7_1_179UDI-idt-UMI
1,HL3LCDSX7,2,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NB,HL3LCDSX7_2_179UDI-idt-UMI
2,HL3LCDSX7,3,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NB,HL3LCDSX7_3_179UDI-idt-UMI
3,HL3LCDSX7,4,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NB,HL3LCDSX7_4_179UDI-idt-UMI
4,HL3VFDSX7,1,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NA,HL3VFDSX7_1_179UDI-idt-UMI


In [2]:
# Rename columns with no ID or confusing names
metadata_df = metadata_df.rename(columns={'Unnamed: 9':'FASTQ_ID','SAMPLE BARCODE':'SAMPLE_BARCODE'})
metadata_df.head()

,FLOWCELL,LANE,MULTIPLEX INDEX,APPLICATION,LIBRARY BARCODE (internal id),SAMPLE_BARCODE,SAMPLE NAME,SPECIES,MACHINE,FASTQ_ID
0,HL3LCDSX7,1,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NB,HL3LCDSX7_1_179UDI-idt-UMI
1,HL3LCDSX7,2,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NB,HL3LCDSX7_2_179UDI-idt-UMI
2,HL3LCDSX7,3,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NB,HL3LCDSX7_3_179UDI-idt-UMI
3,HL3LCDSX7,4,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NB,HL3LCDSX7_4_179UDI-idt-UMI
4,HL3VFDSX7,1,179UDI-idt-UMI,WG-Seq,0856AI,AX4967,B20-235,Homo sapiens,02NA,HL3VFDSX7_1_179UDI-idt-UMI


In [3]:
#Remove innecessary data of metadata for the sarek input file
sarek_df = metadata_df.drop(['MULTIPLEX INDEX','LIBRARY BARCODE (internal id)','SAMPLE NAME','SPECIES','MACHINE','APPLICATION'], axis=1)

# Modify LANE so it contains the number of the flowcell 
sarek_df['LANE']='_' + sarek_df['LANE'].astype(str)
sarek_df['lane'] = sarek_df['FLOWCELL'] + sarek_df['LANE']

# Insert the columns related to information of the patient
patient = 'patient1'
sarek_df['PATIENT'] = patient
sarek_df.head()

,FLOWCELL,LANE,SAMPLE_BARCODE,FASTQ_ID,lane,PATIENT
0,HL3LCDSX7,_1,AX4967,HL3LCDSX7_1_179UDI-idt-UMI,HL3LCDSX7_1,patient1
1,HL3LCDSX7,_2,AX4967,HL3LCDSX7_2_179UDI-idt-UMI,HL3LCDSX7_2,patient1
2,HL3LCDSX7,_3,AX4967,HL3LCDSX7_3_179UDI-idt-UMI,HL3LCDSX7_3,patient1
3,HL3LCDSX7,_4,AX4967,HL3LCDSX7_4_179UDI-idt-UMI,HL3LCDSX7_4,patient1
4,HL3VFDSX7,_1,AX4967,HL3VFDSX7_1_179UDI-idt-UMI,HL3VFDSX7_1,patient1


In [4]:
# Reorder the columns
sarek_df = sarek_df[['PATIENT','SAMPLE_BARCODE', 'lane', 'FASTQ_ID']]

# Add column status: 0 is the reference sample, 1 for tumor samples
status ={'AB9766':'0','AB9767':'1','AX4967':'1'}
sarek_df['status']=sarek_df['SAMPLE_BARCODE'].map(status)
sarek_df.head()

,PATIENT,SAMPLE_BARCODE,lane,FASTQ_ID,status
0,patient1,AX4967,HL3LCDSX7_1,HL3LCDSX7_1_179UDI-idt-UMI,1
1,patient1,AX4967,HL3LCDSX7_2,HL3LCDSX7_2_179UDI-idt-UMI,1
2,patient1,AX4967,HL3LCDSX7_3,HL3LCDSX7_3_179UDI-idt-UMI,1
3,patient1,AX4967,HL3LCDSX7_4,HL3LCDSX7_4_179UDI-idt-UMI,1
4,patient1,AX4967,HL3VFDSX7_1,HL3VFDSX7_1_179UDI-idt-UMI,1


In [5]:
# Add information about the path with new columns
ruta2 = '/workspace/datasafe/sjd_seq/20231006/FASTQ/'
sarek_df['PATH'] = ruta2

# Then reorder
sarek_df = sarek_df[['PATIENT','SAMPLE_BARCODE', 'lane','PATH','FASTQ_ID','status']]

# Add a new column with the file information related to the Fw/Rv file name
F_values = ['_1.fastq.gz'] * len(sarek_df) # we need that this value is applied in all the rows until the end
R_values = ['_2.fastq.gz'] * len(sarek_df)

sarek_df['F_values'] = F_values # add it into a new column
sarek_df['R_values'] = R_values

# Join the two columns of the dataframe corresponding to path and F/R route.
sarek_df['FASTQ_ID_F'] = sarek_df['FASTQ_ID'].astype(str) + sarek_df['F_values']
sarek_df['FASTQ_ID_R'] = sarek_df['FASTQ_ID'].astype(str) + sarek_df['R_values']

# Join the file names
sarek_df['PATH_FASTQ_F'] = sarek_df['PATH'] + sarek_df['FASTQ_ID_F']
sarek_df['PATH_FASTQ_R'] = sarek_df['PATH'] + sarek_df['FASTQ_ID_R']
sarek_df.head()

,PATIENT,SAMPLE_BARCODE,lane,PATH,FASTQ_ID,status,F_values,R_values,FASTQ_ID_F,FASTQ_ID_R,PATH_FASTQ_F,PATH_FASTQ_R
0,patient1,AX4967,HL3LCDSX7_1,/workspace/datasafe/sjd_seq/20231006/FASTQ/,HL3LCDSX7_1_179UDI-idt-UMI,1,_1.fastq.gz,_2.fastq.gz,HL3LCDSX7_1_179UDI-idt-UMI_1.fastq.gz,HL3LCDSX7_1_179UDI-idt-UMI_2.fastq.gz,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
1,patient1,AX4967,HL3LCDSX7_2,/workspace/datasafe/sjd_seq/20231006/FASTQ/,HL3LCDSX7_2_179UDI-idt-UMI,1,_1.fastq.gz,_2.fastq.gz,HL3LCDSX7_2_179UDI-idt-UMI_1.fastq.gz,HL3LCDSX7_2_179UDI-idt-UMI_2.fastq.gz,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
2,patient1,AX4967,HL3LCDSX7_3,/workspace/datasafe/sjd_seq/20231006/FASTQ/,HL3LCDSX7_3_179UDI-idt-UMI,1,_1.fastq.gz,_2.fastq.gz,HL3LCDSX7_3_179UDI-idt-UMI_1.fastq.gz,HL3LCDSX7_3_179UDI-idt-UMI_2.fastq.gz,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
3,patient1,AX4967,HL3LCDSX7_4,/workspace/datasafe/sjd_seq/20231006/FASTQ/,HL3LCDSX7_4_179UDI-idt-UMI,1,_1.fastq.gz,_2.fastq.gz,HL3LCDSX7_4_179UDI-idt-UMI_1.fastq.gz,HL3LCDSX7_4_179UDI-idt-UMI_2.fastq.gz,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
4,patient1,AX4967,HL3VFDSX7_1,/workspace/datasafe/sjd_seq/20231006/FASTQ/,HL3VFDSX7_1_179UDI-idt-UMI,1,_1.fastq.gz,_2.fastq.gz,HL3VFDSX7_1_179UDI-idt-UMI_1.fastq.gz,HL3VFDSX7_1_179UDI-idt-UMI_2.fastq.gz,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...


In [6]:
# Now you can delete columns used to create column PATH_FASTQ_F and PATH_FASTQ_Rv
sarek_df=sarek_df.drop(['PATH','FASTQ_ID_R','F_values','R_values','FASTQ_ID','FASTQ_ID_F','FASTQ_ID_R'],axis=1)

# Rename columns according to SAREK parameters: "patient,sample,lane,fastq_1,fastq_2"
sarek_df=sarek_df.rename(columns={'PATIENT': 'patient', 'SAMPLE_BARCODE':'sample', 'PATH_FASTQ_F':'fastq_1','PATH_FASTQ_R':'fastq_2'})
sarek_df.head()

,patient,sample,lane,status,fastq_1,fastq_2
0,patient1,AX4967,HL3LCDSX7_1,1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
1,patient1,AX4967,HL3LCDSX7_2,1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
2,patient1,AX4967,HL3LCDSX7_3,1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
3,patient1,AX4967,HL3LCDSX7_4,1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
4,patient1,AX4967,HL3VFDSX7_1,1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...


In [7]:
# Check if there are errors in the path
print('Path:', sarek_df.iloc[0,4])

# Check if there are errors in the status/sample
print(sarek_df.iloc[9])
print('Table dimensions:',sarek_df.shape)

Path: /workspace/datasafe/sjd_seq/20231006/FASTQ/HL3LCDSX7_1_179UDI-idt-UMI_1.fastq.gz
patient                                             patient1
sample                                                AB9767
lane                                             HL3VYDSX7_1
status                                                     1
fastq_1    /workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
fastq_2    /workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
Name: 9, dtype: object
Table dimensions: (28, 6)


In [8]:
# Add column Sex:
Sex = ['XY'] * len(sarek_df)
sarek_df['sex'] = Sex # add it into a new column

# Reorder column and add a variable status:
sarek_df = sarek_df[['patient','sex','status','sample','lane','fastq_1','fastq_2']]

# Add column status
sarek_df['sample']
sarek_df

,patient,sex,status,sample,lane,fastq_1,fastq_2
0,patient1,XY,1,AX4967,HL3LCDSX7_1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
1,patient1,XY,1,AX4967,HL3LCDSX7_2,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
2,patient1,XY,1,AX4967,HL3LCDSX7_3,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
3,patient1,XY,1,AX4967,HL3LCDSX7_4,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
4,patient1,XY,1,AX4967,HL3VFDSX7_1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
5,patient1,XY,1,AX4967,HL3VFDSX7_2,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
6,patient1,XY,1,AX4967,HL3VFDSX7_3,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
7,patient1,XY,1,AX4967,HL3VFDSX7_4,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
8,patient1,XY,0,AB9766,HL3VYDSX7_1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...
9,patient1,XY,1,AB9767,HL3VYDSX7_1,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...,/workspace/datasafe/sjd_seq/20231006/FASTQ/HL3...


In [9]:
# Check final tab is correct according to input structure
sarek_df.groupby(['patient','sample','lane']).count()
sarek_df.shape
sarek_df[(sarek_df['patient']=='patient1')&(sarek_df['sample']=='AX4967')].count()

patient    12
sex        12
status     12
sample     12
lane       12
fastq_1    12
fastq_2    12
dtype: int64

In [10]:
# Export tab to CSV
# sarek_df.to_csv('/workspace/datasets/sjd_melos/sarek_results/input.csv',index=False)